In [1]:
import sys
sys.path.append('..')

import pandas as pd
from data_center.binance_handler_v2 import BinanceHandHandler

In [6]:
BinanceHandHandler().endpoint_list

{'fundingrate': '/fapi/v1/fundingRate',
 'continuousklines': 'fapi/v1/continuousKlines'}

In [50]:
import requests
import pytz
import time
from datetime import datetime, timedelta

timezone = pytz.timezone("Asia/Hong_Kong")

BASE_URL = "https://fapi.binance.com"
FUNDING_RATE_ENDPOINT = "/fapi/v1/fundingRate"

symbol = "BTCUSDT"

start_time = int(pd.Timestamp("2016-01-01 00:00:00", tz = "Asia/Hong_Kong" ).timestamp() * 1000)

yesterday = datetime.now() - timedelta(days=1)
yesterday_timestamp_ms = int(yesterday.timestamp() * 1000)
end_time = yesterday_timestamp_ms

df = pd.DataFrame(columns=['fundingTime', 'fundingRate'])

while start_time < end_time:
    response = requests.get(BASE_URL + FUNDING_RATE_ENDPOINT, params={
        "symbol": symbol,
        "startTime": start_time,
        "endTime": end_time,
        "limit" : 1000,
    })

    # Checking if the request was successful
    if response.status_code == 200:
        # Parsing the response to JSON
        data = response.json()

        new_df = pd.DataFrame(data)

        new_df['fundingTime'] = pd.to_datetime(new_df['fundingTime'], unit='ms')#, utc=True)
        # its timezone is Asia even I use utc = True ...

        new_df['fundingTime'] = new_df['fundingTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
        new_df = new_df.drop('symbol',axis =1)

        df = pd.concat([df, new_df],ignore_index=True)
        # The columns "markPrice" = Close as your timezone

    else:
        print("Failed to fetch data: Status code", response.status_code)


In [ ]:
df

In [49]:
new_df

,fundingTime,fundingRate,markPrice
0,2023-12-31 16:00:00,0.00033597,42489.70000000
1,2024-01-01 00:00:00,0.00037409,42313.90000000
2,2024-01-01 08:00:00,0.00027213,42525.11019858
3,2024-01-01 16:00:00,0.00033601,42811.29637234
4,2024-01-02 00:00:00,0.00065846,44230.20000000
...,...,...,...
324,2024-04-17 16:00:00,0.00006534,60280.74173759
325,2024-04-18 00:00:00,0.00006937,61254.50000000
326,2024-04-18 08:00:00,0.00005982,61154.70000000
327,2024-04-18 16:00:00,-0.00003334,63474.00000000


In [40]:
Close = BinanceHandHandler().get_factor_data('close')
Close

,BTCUSDT,ETHUSDT,BNBUSDT,AAVEUSDT,AVAXUSDT,ETCUSDT,SOLUSDT,ADAUSDT,LINKUSDT,BCHUSDT,...,UNIUSDT,XRPUSDT,XLMUSDT,MATICUSDT,TRXUSDT,DOGEUSDT,INJUSDT,DASHUSDT,XEMUSDT,ZECUSDT
datetime,,,,,,,,,,,,,,,,,,,,,
2019-09-08,10391.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-09,10307.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-10,10102.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-11,10159.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-12,10415.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-15,63392.70,3100.75,552.18,84.82,35.136,26.431,138.442,0.4606,13.573,507.52,...,7.243,0.4973,0.10755,0.7074,0.11141,0.16115,24.430,28.41,0.0362,21.21
2024-04-16,63765.00,3082.85,537.52,86.65,34.734,26.160,136.183,0.4581,13.456,486.07,...,7.182,0.4960,0.10954,0.7026,0.11177,0.15594,25.012,28.17,0.0358,21.39
2024-04-17,61254.50,2983.25,533.94,83.61,33.550,25.387,131.781,0.4438,13.124,463.33,...,6.885,0.4947,0.10754,0.6674,0.10956,0.14747,26.012,27.57,0.0349,21.03
